# Configuração da API Key e Cliente Gemini

In [22]:
# -*- coding: utf-8 -*-
"""
Descomplica Web - Projeto Final Imersão IA Alura + Google Gemini
Versão com tom de voz equilibrado, uso mínimo de emojis e interface Gradio.
"""

# Instalações (executar uma vez)
!pip -q install google-genai
!pip -q install google-adk
!pip -q install gradio

import os
from google.colab import userdata # Para Colab
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types as genai_types
import textwrap
from IPython.display import display, Markdown
import warnings
import gradio as gr

warnings.filterwarnings("ignore")

In [23]:
try:
    # Tenta obter a API Key do userdata do Colab
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except Exception as e:
    # Mensagem para o caso de não estar no Colab ou falha no userdata
    print(f"Aviso: Não foi possível obter GOOGLE_API_KEY automaticamente. Erro: {e}")
    print("Se não estiver usando o Google Colab, certifique-se de que a variável de ambiente GOOGLE_API_KEY está configurada.")

# Verificação crítica da API Key
if not os.environ.get("GOOGLE_API_KEY"):
    raise ValueError("API Key do Google não encontrada. "
                     "No Google Colab, configure-a em 'Segredos'. "
                     "Em outros ambientes, defina a variável de ambiente GOOGLE_API_KEY.")

client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Funções Auxiliares

In [24]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    # Usar um ID de sessão consistente para esta versão
    session = session_service.create_session(app_name=agent.name, user_id="user_descomplica_web", session_id="session_dw_v3")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = genai_types.Content(role="user", parts=[genai_types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user_descomplica_web", session_id="session_dw_v3", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
    return final_response.strip()

def to_markdown(text):
  text = text.replace('•', '  *') # Mantém a formatação de bullets
  text = text.replace('\n', '  \n') # Mantém quebras de linha do Markdown
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Definição dos Agentes Especialistas

In [25]:
# Agente que fornece informações sobre o 'Descomplica Web'.
def agente_sobre_o_bot(pergunta_usuario: str) -> str:

    agent_sobre_instance = Agent(
        name="AgenteSobreOBot",
        model=MODEL_ID,
        instruction=f"""
            Olá! Eu sou o 'Descomplica Web', um assistente digital desenvolvido para ajudar jovens
            a entenderem melhor o universo da tecnologia e da internet.
            Minha função é explicar conceitos, tirar dúvidas e oferecer orientações sobre:
            * Criação e gerenciamento de senhas seguras.
            * Componentes e funcionamento de URLs (endereços da web).
            * Dicas para realizar pesquisas eficientes no Google.
            * Diferentes formatos de arquivo (como PDF, JPG, DOCX) e seus usos.
            * Termos técnicos comuns em informática e internet.
            * Práticas de segurança online e cidadania digital.

            Estou aqui para tornar a tecnologia mais acessível e compreensível.
            Considerando a pergunta: '{pergunta_usuario}', forneça uma resposta clara e informativa sobre minhas capacidades.
            """,
        description="Agente que explica o propósito e as funcionalidades do 'Descomplica Web'."
    )
    return call_agent(agent_sobre_instance, pergunta_usuario)

In [26]:
# Agente especialista em senhas.
def agente_senhas(pergunta_usuario: str) -> str:
    agent_senhas_instance = Agent(
        name="AgenteDeSenhas",
        model=MODEL_ID,
        instruction=f"""
            Olá! Sou o AgenteDeSenhas do sistema 'Descomplica Web'.
            Estou aqui para ajudar você com questões sobre senhas: como criar senhas fortes,
            entender a importância da segurança de senhas e como proteger suas contas online.
            Meu objetivo é explicar esses tópicos de forma clara e objetiva.
            Respondendo à sua pergunta: '{pergunta_usuario}'
            """,
        description="Agente que oferece orientação sobre segurança e criação de senhas."
    )
    return call_agent(agent_senhas_instance, pergunta_usuario)

In [27]:
# Agente especialista em URLs.
def agente_urls(pergunta_usuario: str) -> str:
    agent_urls_instance = Agent(
        name="AgenteDeURLs",
        model=MODEL_ID,
        instruction=f"""
            Olá! Sou o AgenteDeURLs do 'Descomplica Web'. Minha especialidade é ajudar a entender
            os endereços da web (URLs). Posso explicar o que são, como são estruturados
            e o que cada parte de uma URL significa.
            Se você tem dúvidas sobre uma URL específica ou sobre o conceito geral, pode perguntar.
            Analisando sua pergunta: '{pergunta_usuario}'
            """,
        description="Agente que explica a estrutura e os componentes de URLs."
    )
    return call_agent(agent_urls_instance, pergunta_usuario)

In [28]:
# Agente especialista em buscas no Google.
def agente_buscas(pergunta_usuario: str) -> str:
    agent_buscas_instance = Agent(
        name="AgenteDeBuscas",
        model=MODEL_ID,
        instruction=f"""
            Olá! Sou o AgenteDeBuscas do 'Descomplica Web'.
            Posso ajudar você a melhorar suas habilidades de pesquisa no Google,
            oferecendo dicas sobre como usar operadores de busca (como aspas, 'site:', 'filetype:')
            para encontrar informações de forma mais precisa.
            Se precisar, também posso realizar uma busca sobre um tópico específico usando a ferramenta 'google_search'.
            Vamos explorar sua necessidade de busca: '{pergunta_usuario}'
            """,
        description="Agente que auxilia com técnicas de pesquisa no Google e pode realizar buscas.",
        tools=[google_search]
    )
    return call_agent(agent_buscas_instance, pergunta_usuario)

In [29]:
#Agente especialista em formatos de arquivo.
def agente_formatos(pergunta_usuario: str) -> str:
    agent_formatos_instance = Agent(
        name="AgenteDeFormatos",
        model=MODEL_ID,
        instruction=f"""
            Olá! Eu sou o AgenteDeFormatos do 'Descomplica Web'.
            Minha função é explicar sobre os diferentes formatos de arquivo que encontramos no dia a dia digital,
            como .PDF, .JPG, .DOCX, .MP3, .ZIP, entre outros.
            Posso esclarecer para que servem, suas principais características e como são utilizados.
            Importante: eu não realizo conversões de arquivos, mas posso explicar os métodos comuns para isso.
            Qual sua dúvida sobre formatos de arquivo? '{pergunta_usuario}'
            """,
        description="Agente que explica sobre diversos formatos de arquivo e seus usos."
    )
    return call_agent(agent_formatos_instance, pergunta_usuario)

In [30]:
# Agente para esclarecer termos técnicos.
def agente_tira_duvidas_tech(pergunta_usuario: str) -> str:
    agent_tira_duvidas_instance = Agent(
        name="AgenteTiraDuvidasTech",
        model=MODEL_ID,
        instruction=f"""
            Olá! Sou o AgenteTiraDuvidasTech do 'Descomplica Web'.
            Se você encontrou algum termo técnico de informática, internet ou tecnologia que não compreendeu bem,
            estou aqui para ajudar a esclarecer. Meu objetivo é explicar esses termos
            de forma simples e direta, para facilitar seu entendimento.
            Qual termo ou conceito você gostaria de entender melhor? '{pergunta_usuario}'
            """,
        description="Agente que explica termos técnicos e conceitos de tecnologia de forma clara."
    )
    return call_agent(agent_tira_duvidas_instance, pergunta_usuario)

In [31]:
# Agente especialista em cidadania digital e segurança online.
def agente_cidadania_digital(pergunta_usuario: str) -> str:
    agent_cidadania_digital_instance = Agent(
        name="AgenteCidadaniaDigital",
        model=MODEL_ID,
        instruction=f"""
            Olá! Sou o AgenteCidadaniaDigital do 'Descomplica Web'.
            Minha área é orientar sobre segurança online e práticas de cidadania digital.
            Isso inclui como se proteger de ameaças virtuais, identificar notícias falsas (fake news),
            a importância da privacidade online e como ter um comportamento ético e respeitoso na internet.
            Como posso ajudar com sua dúvida sobre este tema? '{pergunta_usuario}'
            """,
        description="Agente que oferece orientação sobre segurança online e comportamento ético na internet."
    )
    return call_agent(agent_cidadania_digital_instance, pergunta_usuario)

# Definição do Agente Roteador

In [32]:
# Agente que analisa a pergunta do usuário e direciona para o especialista correto.
def agente_roteador(pergunta_usuario: str) -> str:
    nomes_agentes = [
        "AgenteSobreOBot",
        "AgenteDeSenhas",
        "AgenteDeURLs",
        "AgenteDeBuscas",
        "AgenteDeFormatos",
        "AgenteTiraDuvidasTech",
        "AgenteCidadaniaDigital"
    ]
    nomes_agentes_str = ", ".join([f"'{nome}'" for nome in nomes_agentes])

    roteador_instance = Agent(
        name="AgenteRoteadorDescomplicaWeb",
        model=MODEL_ID,
        instruction=f"""
            Você é o sistema de roteamento do 'Descomplica Web', um assistente focado em ajudar jovens
            com questões de tecnologia. Sua tarefa é analisar a pergunta do usuário e encaminhá-la
            ao agente especialista mais adequado.

            Os especialistas disponíveis são:
            - 'AgenteSobreOBot': Para perguntas sobre o próprio assistente 'Descomplica Web' (o que ele faz, suas capacidades). Ex: "O que você é capaz de fazer?", "Quem é você?"
            - 'AgenteDeSenhas': Para dúvidas sobre criação, segurança e gerenciamento de senhas. Ex: "Como criar uma senha forte?", "minha senha 'abc123' é segura?"
            - 'AgenteDeURLs': Para explicar partes de um endereço web (URL). Ex: "O que significa 'https' em um link?", "analise a URL: exemplo.com/pagina"
            - 'AgenteDeBuscas': Para dicas sobre como pesquisar no Google e realizar buscas de informações. Ex: "Como encontrar informações sobre história do Brasil?", "dicas para pesquisa online"
            - 'AgenteDeFormatos': Para explicar tipos de arquivos (PDF, JPG, etc.) e questões relacionadas a formatos. Ex: "O que é um arquivo .RAR?", "para que serve o formato .PNG?"
            - 'AgenteTiraDuvidasTech': Para explicar termos técnicos e conceitos básicos de informática/internet. Ex: "O que é 'cookies' de um site?", "o que significa 'firewall'?"
            - 'AgenteCidadaniaDigital': Para conselhos sobre segurança online, fake news e comportamento ético na internet. Ex: "Como identificar uma notícia falsa?", "é seguro compartilhar dados pessoais em jogos?"

            Analise a pergunta do usuário: '{pergunta_usuario}'

            Responda APENAS com o NOME EXATO de um dos seguintes agentes especialistas: {nomes_agentes_str}.
            Se a pergunta não se encaixar claramente em nenhum especialista, for muito vaga, ou não estiver relacionada aos tópicos que os agentes cobrem, responda 'NaoSeiQualAgente'.
            Selecione o agente que melhor corresponde à intenção principal da pergunta.
            """,
        description="Agente que direciona a pergunta do usuário para o especialista correto, com tom equilibrado."
    )
    nome_do_agente_especialista = call_agent(roteador_instance, f"Analisar pergunta para roteamento: {pergunta_usuario}")
    if nome_do_agente_especialista in nomes_agentes or nome_do_agente_especialista == "NaoSeiQualAgente":
        return nome_do_agente_especialista
    else:
        # Log interno para o desenvolvedor, caso o Gemini não siga a instrução
        print(f"Alerta de Roteamento: AgenteRoteador retornou um nome inesperado: '{nome_do_agente_especialista}'. Padronizando para NaoSeiQualAgente.")
        return "NaoSeiQualAgente"

# Lógica do Chatbot para a Interface Gradio

In [33]:
def chatbot_interface(pergunta_usuario: str) -> str:
    """
    Função principal que processa a pergunta do usuário e retorna a resposta do bot.
    Esta função será usada pela interface Gradio.
    """
    if not pergunta_usuario or pergunta_usuario.strip() == "":
        return "Por favor, digite uma pergunta válida."

    # Logs no console (opcional, mas útil para debug no Colab)
    print(f"\n[Gradio Input] Pergunta recebida: '{pergunta_usuario}'")
    try:
        especialista_escolhido = agente_roteador(pergunta_usuario)
        print(f"[Gradio Logic] Especialista escolhido: {especialista_escolhido}")

        resposta_final = ""
        mapa_agentes = {
            "AgenteSobreOBot": agente_sobre_o_bot,
            "AgenteDeSenhas": agente_senhas,
            "AgenteDeURLs": agente_urls,
            "AgenteDeBuscas": agente_buscas,
            "AgenteDeFormatos": agente_formatos,
            "AgenteTiraDuvidasTech": agente_tira_duvidas_tech,
            "AgenteCidadaniaDigital": agente_cidadania_digital,
        }

        if especialista_escolhido in mapa_agentes:
            resposta = mapa_agentes[especialista_escolhido](pergunta_usuario)
            resposta_final = resposta.replace("**", "")
        elif especialista_escolhido == "NaoSeiQualAgente":
            resposta_final = (
                "Desculpe, não consegui direcionar sua pergunta para um especialista específico ou "
                "o tema parece estar fora do meu escopo atual.\n"
                "Você pode tentar reformular sua pergunta ou perguntar sobre os seguintes tópicos:\n"
                "  * Criação e segurança de senhas\n"
                "  * Entendimento de endereços da web (URLs)\n"
                "  * Dicas para pesquisas online\n"
                "  * Formatos de arquivo (PDF, JPG, etc.)\n"
                "  * Explicação de termos de tecnologia\n"
                "  * Segurança e comportamento na internet\n"
                "  * O que eu sou e o que posso fazer."
            )
        else:
            resposta_final = "Ocorreu um erro ao tentar processar sua pergunta. Por favor, tente novamente."
            print(f"[Gradio Error] Especialista '{especialista_escolhido}' não reconhecido.")

        print(f"[Gradio Output] Resposta gerada (primeiros 100 chars): {resposta_final[:100]}")
        return resposta_final if resposta_final else "Não foi possível gerar uma resposta para esta pergunta."

    except Exception as e:
        print(f"[Gradio Error] Exceção durante o processamento: {e}")
        # Retornar uma mensagem de erro amigável para a interface
        return f"Ocorreu um erro ao processar sua pergunta: {str(e)}. Por favor, tente novamente."

In [35]:
if __name__ == "__main__":
    if not os.environ.get("GOOGLE_API_KEY"):
        print("ERRO CRÍTICO: A API Key do Google (GOOGLE_API_KEY) não está configurada.")
        print("No Google Colab, certifique-se de que ela está definida na seção 'Segredos'.")
        print("Em outros ambientes, configure a variável de ambiente correspondente.")
        print("O programa não pode continuar sem a API Key.")
    else:
        print("API Key encontrada. Iniciando a interface do Descomplica Web com Gradio...")
        print("Aguarde alguns instantes para a interface carregar e o link público ser gerado (se estiver no Colab).")

        # Define a interface Gradio
        iface = gr.Interface(
            fn=chatbot_interface,
            allow_flagging="never",
            inputs=gr.Textbox(label="Pergunta", placeholder="Digite sua pergunta sobre tecnologia aqui... Pressione Enter ou clique em 'Enviar'.", lines=3,),
            outputs=gr.Textbox(label="Resposta do Descomplica Web", lines=10, interactive=False),
            title="🤖 Descomplica Web 🤖",
            description=(
                "Seu assistente para tirar dúvidas sobre senhas, URLs, buscas na internet, formatos de arquivo, termos técnicos e segurança online.\n"
                "Feito com IA Google Gemini e Agentes Google ADK.\n\n"
                "Para limpar a conversa e começar de novo, use o botão 'Clear'.\n"
                "Para sair da aplicação, simplesmente feche esta aba ou janela do navegador."
            ),
            examples=[
                ["Quem é você e o que você faz?"],
                ["Como faço para criar uma senha realmente forte?"],
                ["O que significa HTTPS em um endereço de site?"],
                ["Como posso achar informações sobre a história da programação para um trabalho da escola?"],
                ["O que é um arquivo PDF e para que ele serve?"],
                ["O que quer dizer 'phishing'?"],
                ["É seguro usar redes Wi-Fi públicas?"]
            ],
            theme=gr.themes.Soft()
        )

        # Inicia a interface
        # debug=True é útil para ver logs do Gradio no console.
        try:
            iface.launch(share=True, debug=True)
            print("Interface Gradio iniciada. Verifique o output acima para o link local e o link público (se share=True).")
        except Exception as e:
            print(f"Erro ao iniciar a interface Gradio: {e}")
            print("Verifique se a porta 7860 (padrão do Gradio) está disponível ou tente reiniciar o kernel do Colab.")

API Key encontrada. Iniciando a interface do Descomplica Web com Gradio...
Aguarde alguns instantes para a interface carregar e o link público ser gerado (se estiver no Colab).
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://74800e1ce059204dd4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



[Gradio Input] Pergunta recebida: 'Como posso achar informações sobre a história da programação para um trabalho da escola?'
[Gradio Logic] Especialista escolhido: AgenteDeBuscas
[Gradio Output] Resposta gerada (primeiros 100 chars): Para encontrar informações sobre a história da programação para o seu trabalho escolar, você pode se
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://74800e1ce059204dd4.gradio.live
Interface Gradio iniciada. Verifique o output acima para o link local e o link público (se share=True).
